# Problemset 4: Optimization
## Assignment:

1. Select only male heads of the household between 25 and 60 years of age that earn > \$7/hr.
2. Create indicator and continuous variables.
3. Estimate the following model via a maximum likelihood estimator separately for t = 1971, 1980, 1990, and 2000.

$ln(w_{i,t}) = \alpha + \beta_{1}Educ_{i,t} + \beta_{2}Age_{i,t} + \beta_{3}Age^{2}_{i,t} + \beta_{4}Black_{i,t} + \beta_{5}Hispanic_{i,t} + \beta_{6}OtherRace_{i,t} + \epsilon_{i,t}$

4. Interpret the coefficient $\beta_{1}$. How do the returns to education change over time in these data?

## The Data

In [1]:
# importing packages
import pandas as pd
import numpy as np
import scipy.optimize as opt
import statsmodels.api as sm
import math
import time

In [2]:
# read in the data
data = pd.read_stata('PS4_data.dta')

# becoming familiar with the data
data.head(n=5)

,id68,year,intid,relhh,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,...,redpregovinc,hsex,wsex,age,wage,hpersno,wpersno,hyrsed,wyrsed,pce
0,1,1967,1,Head,1200.0,2000.0,NaN,NaN,0,NaN,...,5614.0,1.0,2.0,52.0,46.0,1.0,2.0,8.0,8.0,0.0
1,2,1967,2,Head,0.0,0.0,NaN,NaN,0,NaN,...,0.0,1.0,2.0,56.0,57.0,1.0,2.0,3.0,3.0,0.0
2,3,1967,3,Head,0.0,0.0,NaN,NaN,0,NaN,...,0.0,1.0,2.0,77.0,64.0,1.0,2.0,NaN,3.0,0.0
3,4,1967,4,Head,1560.0,0.0,NaN,NaN,6,1.0,...,3280.0,1.0,2.0,45.0,44.0,1.0,2.0,8.0,5.0,0.0
4,5,1967,5,Head,2500.0,2000.0,NaN,NaN,3,1.0,...,7900.0,1.0,2.0,24.0,22.0,1.0,2.0,10.0,9.0,0.0


In [3]:
# general descriptive data
data.describe()

,id68,year,intid,hannhrs,wannhrs,hlabinc,wlabinc,nochild,wrace,hrace,...,redpregovinc,hsex,wsex,age,wage,hpersno,wpersno,hyrsed,wyrsed,pce
count,123786.000000,123786.000000,123786.000000,123786.000000,123786.000000,9.023300e+04,48496.000000,123786.000000,90603.000000,123656.000000,...,1.237860e+05,123786.000000,80758.0,123786.000000,80758.000000,123786.000000,80758.000000,122809.000000,80091.000000,123786.000000
mean,1494.639475,1984.831273,3271.379429,1679.269897,633.026917,4.211505e+04,22026.289062,0.843771,1.098220,1.129731,...,3.012258e+04,1.233072,2.0,45.545547,41.390785,39.620201,55.346169,12.666091,12.720081,0.557690
std,838.901790,9.836212,2277.056058,1061.704712,878.422791,4.670424e+04,21336.107422,1.182829,0.356161,0.394627,...,4.588795e+04,0.422940,0.0,17.623671,14.786721,69.003265,77.864296,2.917721,2.422607,0.265198
min,1.000000,1967.000000,1.000000,0.000000,0.000000,6.353981e-01,1.192780,0.000000,1.000000,1.000000,...,-1.324040e+05,1.000000,2.0,16.000000,13.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,772.000000,1977.000000,1444.000000,832.000000,0.000000,1.979858e+04,8016.247070,0.000000,1.000000,1.000000,...,7.700000e+03,1.000000,2.0,31.000000,29.000000,1.000000,2.000000,12.000000,12.000000,0.362158
50%,1517.000000,1985.000000,2984.000000,1976.000000,0.000000,3.460022e+04,18122.412109,0.000000,1.000000,1.000000,...,1.900000e+04,1.000000,2.0,42.000000,39.000000,3.000000,3.000000,12.000000,12.000000,0.599887
75%,2224.000000,1993.000000,4763.000000,2350.000000,1454.000000,5.267309e+04,30256.060547,2.000000,1.000000,1.000000,...,3.910775e+04,1.000000,2.0,58.000000,51.000000,22.000000,170.000000,15.000000,14.000000,0.786908
max,2930.000000,2002.000000,16968.000000,7800.000000,5840.000000,3.771521e+06,856942.062500,11.000000,8.000000,8.000000,...,3.660000e+06,2.000000,2.0,102.000000,95.000000,227.000000,231.000000,17.000000,17.000000,0.928007


## Cleaning & preprocessing the data

In [4]:
#Choose only those observations that have male heads of households, are between 25 and 60 years of age 
#and earn wages > $7/hr

data = data[(data['hannhrs'] > 0)]
data['hrlywage'] = data['hlabinc'] / data['hannhrs']
data = data[(data['hsex'] == 1) & 
            (data['age'] >= 25) & 
            (data['age'] <= 60) & 
            (data['hrlywage'] >= 7)]

In [5]:
#Create race dummy variables

data['black'] = (data['hrace'] == 2).astype(int)
data['hispanic'] = (data['hrace'] == 5).astype(int)
data['other'] = ((data['hrace'] == 3) | 
                 (data['hrace'] == 4) | 
                 (data['hrace'] == 6) | 
                 (data['hrace'] == 7)).astype(int)


#Create log of hourly wage (dependent variable)

data['lnwage'] = np.log(data['hrlywage'])

#Keep only the needed variables - log of wages, education, age, race dummies

data = data[['year','lnwage','hyrsed','age','black','hispanic','other']]

data.describe()

,year,lnwage,hyrsed,age,black,hispanic,other
count,57477.000000,57477.000000,57097.000000,57477.000000,57477.000000,57477.0,57477.000000
mean,1986.635245,3.010414,13.529993,39.224247,0.056388,0.0,0.023035
std,8.744894,0.543891,2.449510,9.579065,0.230671,0.0,0.150017
min,1971.000000,1.945946,1.000000,25.000000,0.000000,0.0,0.000000
25%,1979.000000,2.635309,12.000000,31.000000,0.000000,0.0,0.000000
50%,1987.000000,2.990979,13.000000,38.000000,0.000000,0.0,0.000000
75%,1994.000000,3.324576,16.000000,47.000000,0.000000,0.0,0.000000
max,2002.000000,7.448526,17.000000,60.000000,1.000000,0.0,1.000000


$\color{red}{\text{We do not have any hispanics in our dataset.}}$

## The optimizer

We now define the log-likelihood function that needs to be maximized in order to obtain the parameter estimates of the model$\color{red}{*}$.

$$ l(\alpha, \beta) = \ln(\prod\limits_{i=1}^N p(y_i\mid x_i,\alpha,\beta) = \sum\limits_{i=1}^N \ln p(y_i\mid x_i,\alpha,\beta) $$
We assume that $\epsilon_i \sim N(0,\sigma^2)$. This leads to the likelihood function being of the form

$$ \begin{split} l(\alpha, \beta, \sigma^2) &= \sum\limits_{i=1}^N \ln\left[\left(\frac{1}{2\pi\sigma^2}\right)^{1/2}\exp\left(-\frac{1}{2\pi\sigma^2}\left(y_i - \alpha - \beta^T x_i\right)^2\right)\right] \\ &= \frac{N}{2} \ln\left(\frac{1}{2\pi\sigma^2}\right) - \frac{1}{2\sigma^2}\sum\limits_{i=1}^N\left(y_i - \alpha - \beta^T x_i\right)^2 \end{split} $$
We need to find $\alpha$, $\beta$ and $\sigma^2$ that maximize the above function. We begin by defining this function and then use a numerical optimization technique to find these maxima.

$\color{red}{\text{*Taken from outside sources and senior students.}}$

In [6]:
# defining the function
def mle(Beta, yearly_data):
    '''
    Computes the value of the log-likelihood function that we need to maximize
    
    Args:
        Beta: A 7 component tuple for the model parameters (sigmasq, alpha, beta1, beta2, beta3, beta4, beta5)
        yearly_data: Data of the particular year for which the model is to be estimated
        
    Returns:
        ll_val: Value of the log-likelihood function
    '''
    
    sigmasq, alpha, beta1, beta2, beta3, beta4, beta5 = Beta
    
    SSE = ((yearly_data['lnwage'] - alpha - (beta1 * yearly_data['hyrsed'] + 
                                             beta2 * yearly_data['age'] + 
                                             beta3 * yearly_data['age'] ** 2 +
                                             beta4 * yearly_data['black'] + 
                                             beta5 * yearly_data['other'])) ** 2).sum()
    
    n = len(yearly_data)
    
    #Compute the log-likelihood
    #We use the negative log-likelihood since we're call a minimization routine
    
    ll_val = -((n / 2) * (np.log(1 / 2 * np.pi * sigmasq)) - (1 / 2 * sigmasq) * SSE)
    
    return ll_val

## Optimization for the individual years
We optimize for each year individually. I assume there must be a faster way to do this using for loops, but I couldn't figure it out.

Also, I used the Nelder-Mead method rather than the Golden ratio as used in the example because it did not recognize the "Golden" method.

### Optimization for year = 1971

In [7]:
data_71 = data[(data['year'] == 1971)]

#We use the correlation between variables as the initial values for the beta's,
#the variance of the dependent variable as the initial value for sigmasq, and
#the mean of the dependent variable as the initial value for alpha

sig_start = (data_71.loc[:,'lnwage'].std())**2
alpha_start = data_71.loc[:,'lnwage'].mean()

Beta0 = (sig_start, 
         alpha_start, 
         data_71['lnwage'].corr(data_71['hyrsed']),
         data_71['lnwage'].corr(data_71['age']), 
         data_71['lnwage'].corr(data_71['age']), 
         data_71['lnwage'].corr(data_71['black']),
         data_71['lnwage'].corr(data_71['other']))

# Use Nelder-Mead to minimize the negative of the log-likelihood function

min_results_71 = opt.minimize(mle, 
                              Beta0, 
                              args=(data_71),
                              method="Nelder-Mead", 
                              tol=1e-15)

print('The estimates for 1971 are:')
print('alpha = ', min_results_71['x'][1])
print('beta1 = ', min_results_71['x'][2])
print('beta2 = ', min_results_71['x'][3])
print('beta3 = ', min_results_71['x'][4])
print('beta4 = ', min_results_71['x'][5])
print('beta5 = ', min_results_71['x'][6])

The estimates for 1971 are:
alpha =  6.317625760179212
beta1 =  0.05318932744521896
beta2 =  -0.21729546245527204
beta3 =  0.0027329494228654156
beta4 =  0.2958664277444321
beta5 =  -0.002923590962383862


In [8]:
# We use exp(beta1)

math.exp(min_results_71['x'][2])

1.054629296489375

Interpreting the coefficient on $\beta_{1}$

An additional year of education leads to a $\textbf{5.46%}$ increase inwages.

In [9]:
#We comapre the results of our MLE approach with OLS estimates

data_71['const'] = 1
data_71['age_sq'] = data_71['age'] ** 2
var_list = ['const', 'hyrsed','age', 'age_sq', 'black','other']
reg1 = sm.OLS(data_71['lnwage'], data_71[var_list], missing='drop')
results = reg1.fit()
results.params

<ipython-input-9-e855a69dc00f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_71['const'] = 1
<ipython-input-9-e855a69dc00f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_71['age_sq'] = data_71['age'] ** 2


const     0.586331
hyrsed    0.066500
age       0.064904
age_sq   -0.000617
black    -0.164137
other     0.017512
dtype: float64

In [10]:
print('The absolute difference between the estimates for alpha for 1971 is ',
      abs(min_results_71['x'][1] - results.params[0]))
print('The absolute difference between the estimates for beta1 for 1971 is ',
      abs(min_results_71['x'][2] - results.params[1]))
print('The absolute difference between the estimates for beta2 for 1971 is ',
      abs(min_results_71['x'][3] - results.params[2]))
print('The absolute difference between the estimates for beta3 for 1971 is ',
      abs(min_results_71['x'][4] - results.params[3]))
print('The absolute difference between the estimates for beta4 for 1971 is ',
      abs(min_results_71['x'][5] - results.params[4]))
print('The absolute difference between the estimates for beta5 for 1971 is ',
      abs(min_results_71['x'][6] - results.params[5]))

The absolute difference between the estimates for alpha for 1971 is  5.731294546979942
The absolute difference between the estimates for beta1 for 1971 is  0.013310524624629956
The absolute difference between the estimates for beta2 for 1971 is  0.2821998424667791
The absolute difference between the estimates for beta3 for 1971 is  0.0033498902482864477
The absolute difference between the estimates for beta4 for 1971 is  0.46000367640129686
The absolute difference between the estimates for beta5 for 1971 is  0.02043551091917955


### Optimization for year = 1980

In [11]:
data_80 = data[(data['year'] == 1980)]

#We use the correlation between variables as the initial values for the beta's,
#the variance of the dependent variable as the initial value for sigmasq, and
#the mean of the dependent variable as the initial value for alpha

sig_start = (data_80.loc[:,'lnwage'].std())**2
alpha_start = data_80.loc[:,'lnwage'].mean()

Beta0 = (sig_start, 
         alpha_start, 
         data_80['lnwage'].corr(data_80['hyrsed']),
         data_80['lnwage'].corr(data_80['age']), 
         data_80['lnwage'].corr(data_80['age']), 
         data_80['lnwage'].corr(data_80['black']),
         data_80['lnwage'].corr(data_80['other']))

# Use Nelder-Mead to minimize the negative of the log-likelihood function

min_results_80 = opt.minimize(mle, 
                              Beta0, 
                              args=(data_80),
                              method="Nelder-Mead", 
                              tol=1e-15)

print('The estimates for 1980 are:')
print('alpha = ', min_results_80['x'][1])
print('beta1 = ', min_results_80['x'][2])
print('beta2 = ', min_results_80['x'][3])
print('beta3 = ', min_results_80['x'][4])
print('beta4 = ', min_results_80['x'][5])
print('beta5 = ', min_results_80['x'][6])

The estimates for 1980 are:
alpha =  13.221863269805894
beta1 =  0.20500066114597443
beta2 =  -0.6686326563358365
beta3 =  0.008118513105438097
beta4 =  -0.6616156684789034
beta5 =  -0.007032612790667445


In [12]:
# We use exp(beta1)

math.exp(min_results_80['x'][2])

1.2275258765367012

Interpreting the coefficient on $\beta_{1}$

An additional year of education leads to a $\textbf{22.75%}$ increase inwages.

$\color{red}{\textbf{This seems incorrect.}}$

In [13]:
#We comapre the results of our MLE approach with OLS estimates

data_80['const'] = 1
data_80['age_sq'] = data_80['age'] ** 2
var_list = ['const', 'hyrsed','age', 'age_sq', 'black','other']
reg1 = sm.OLS(data_80['lnwage'], data_80[var_list], missing='drop')
results = reg1.fit()
results.params

<ipython-input-13-b55faca9e574>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_80['const'] = 1
<ipython-input-13-b55faca9e574>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_80['age_sq'] = data_80['age'] ** 2


const     1.002272
hyrsed    0.066003
age       0.045569
age_sq   -0.000399
black    -0.103028
other     0.012315
dtype: float64

In [14]:
print('The absolute difference between the estimates for alpha for 1980 is ',
      abs(min_results_80['x'][1] - results.params[0]))
print('The absolute difference between the estimates for beta1 for 1980 is ',
      abs(min_results_80['x'][2] - results.params[1]))
print('The absolute difference between the estimates for beta2 for 1980 is ',
      abs(min_results_80['x'][3] - results.params[2]))
print('The absolute difference between the estimates for beta3 for 1980 is ',
      abs(min_results_80['x'][4] - results.params[3]))
print('The absolute difference between the estimates for beta4 for 1980 is ',
      abs(min_results_80['x'][5] - results.params[4]))
print('The absolute difference between the estimates for beta5 for 1980 is ',
      abs(min_results_80['x'][6] - results.params[5]))

The absolute difference between the estimates for alpha for 1980 is  12.219591263646063
The absolute difference between the estimates for beta1 for 1980 is  0.1389981600759741
The absolute difference between the estimates for beta2 for 1980 is  0.7142011706123349
The absolute difference between the estimates for beta3 for 1980 is  0.008517931637519401
The absolute difference between the estimates for beta4 for 1980 is  0.5585876111062676
The absolute difference between the estimates for beta5 for 1980 is  0.01934756135060701


### Optimization for year = 1990

In [15]:
data_90 = data[(data['year'] == 1990)]

#We use the correlation between variables as the initial values for the beta's,
#the variance of the dependent variable as the initial value for sigmasq, and
#the mean of the dependent variable as the initial value for alpha

sig_start = (data_90.loc[:,'lnwage'].std())**2
alpha_start = data_90.loc[:,'lnwage'].mean()

Beta0 = (sig_start, 
         alpha_start, 
         data_90['lnwage'].corr(data_90['hyrsed']),
         data_90['lnwage'].corr(data_90['age']), 
         data_90['lnwage'].corr(data_90['age']), 
         data_90['lnwage'].corr(data_90['black']),
         data_90['lnwage'].corr(data_90['other']))

# Use Nelder-Mead to minimize the negative of the log-likelihood function

min_results_90 = opt.minimize(mle, 
                              Beta0, 
                              args=(data_90),
                              method="Nelder-Mead", 
                              tol=1e-15)

print('The estimates for 1990 are:')
print('alpha = ', min_results_90['x'][1])
print('beta1 = ', min_results_90['x'][2])
print('beta2 = ', min_results_90['x'][3])
print('beta3 = ', min_results_90['x'][4])
print('beta4 = ', min_results_90['x'][5])
print('beta5 = ', min_results_90['x'][6])

<ipython-input-6-92ce42bf1a6b>:27: RuntimeWarning: invalid value encountered in log
  ll_val = -((n / 2) * (np.log(1 / 2 * np.pi * sigmasq)) - (1 / 2 * sigmasq) * SSE)


The estimates for 1990 are:
alpha =  11.155360697373691
beta1 =  0.041910495925555485
beta2 =  -0.43991285910422784
beta3 =  0.005313906168158591
beta4 =  -0.6453918208531209
beta5 =  0.008977938026799838


In [16]:
# We use exp(beta1)

math.exp(min_results_90['x'][2])

1.0428011396228778

Interpreting the coefficient on $\beta_{1}$

An additional year of education leads to a $\textbf{4.28%}$ increase inwages.

In [17]:
#We comapre the results of our MLE approach with OLS estimates

data_90['const'] = 1
data_90['age_sq'] = data_90['age'] ** 2
var_list = ['const', 'hyrsed','age', 'age_sq', 'black','other']
reg1 = sm.OLS(data_90['lnwage'], data_90[var_list], missing='drop')
results = reg1.fit()
results.params

<ipython-input-17-2d5c9551da95>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_90['const'] = 1
<ipython-input-17-2d5c9551da95>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_90['age_sq'] = data_90['age'] ** 2


const     0.277243
hyrsed    0.095499
age       0.057864
age_sq   -0.000540
black    -0.167988
other    -0.052001
dtype: float64

In [18]:
print('The absolute difference between the estimates for alpha for 1990 is ',
      abs(min_results_90['x'][1] - results.params[0]))
print('The absolute difference between the estimates for beta1 for 1990 is ',
      abs(min_results_90['x'][2] - results.params[1]))
print('The absolute difference between the estimates for beta2 for 1990 is ',
      abs(min_results_90['x'][3] - results.params[2]))
print('The absolute difference between the estimates for beta3 for 1990 is ',
      abs(min_results_90['x'][4] - results.params[3]))
print('The absolute difference between the estimates for beta4 for 1990 is ',
      abs(min_results_90['x'][5] - results.params[4]))
print('The absolute difference between the estimates for beta5 for 1990 is ',
      abs(min_results_90['x'][6] - results.params[5]))

The absolute difference between the estimates for alpha for 1990 is  10.878117535201937
The absolute difference between the estimates for beta1 for 1990 is  0.05358864852471816
The absolute difference between the estimates for beta2 for 1990 is  0.49777696289401846
The absolute difference between the estimates for beta3 for 1990 is  0.005854056824185575
The absolute difference between the estimates for beta4 for 1990 is  0.47740429294423803
The absolute difference between the estimates for beta5 for 1990 is  0.06097861846621253


### Optimization for year = 2000

In [19]:
data_00 = data[(data['year'] == 2000)]

#We use the correlation between variables as the initial values for the beta's,
#the variance of the dependent variable as the initial value for sigmasq, and
#the mean of the dependent variable as the initial value for alpha

sig_start = (data_00.loc[:,'lnwage'].std())**2
alpha_start = data_00.loc[:,'lnwage'].mean()

Beta0 = (sig_start, 
         alpha_start, 
         data_00['lnwage'].corr(data_00['hyrsed']),
         data_00['lnwage'].corr(data_00['age']), 
         data_00['lnwage'].corr(data_00['age']), 
         data_00['lnwage'].corr(data_00['black']),
         data_00['lnwage'].corr(data_00['other']))

# Use Nelder-Mead to minimize the negative of the log-likelihood function

min_results_00 = opt.minimize(mle, 
                              Beta0, 
                              args=(data_00),
                              method="Nelder-Mead", 
                              tol=1e-15)

print('The estimates for 2000 are:')
print('alpha = ', min_results_00['x'][1])
print('beta1 = ', min_results_00['x'][2])
print('beta2 = ', min_results_00['x'][3])
print('beta3 = ', min_results_00['x'][4])
print('beta4 = ', min_results_00['x'][5])
print('beta5 = ', min_results_00['x'][6])

<ipython-input-6-92ce42bf1a6b>:27: RuntimeWarning: invalid value encountered in log
  ll_val = -((n / 2) * (np.log(1 / 2 * np.pi * sigmasq)) - (1 / 2 * sigmasq) * SSE)


The estimates for 2000 are:
alpha =  -7.704632026925449
beta1 =  0.32248604560373495
beta2 =  0.30882655084045696
beta3 =  -0.003592300485871132
beta4 =  0.21473159353690424
beta5 =  -0.04742008204454666


In [20]:
# We use exp(beta1)

math.exp(min_results_00['x'][2])

1.3805556259048883

Interpreting the coefficient on $\beta_{1}$

An additional year of education leads to a $\textbf{38.05%}$ increase inwages.

$\color{red}{\textbf{This seems incorrect.}}$

In [21]:
#We comapre the results of our MLE approach with OLS estimates

data_00['const'] = 1
data_00['age_sq'] = data_00['age'] ** 2
var_list = ['const', 'hyrsed','age', 'age_sq', 'black','other']
reg1 = sm.OLS(data_00['lnwage'], data_00[var_list], missing='drop')
results = reg1.fit()
results.params

<ipython-input-21-8cecf2c6e250>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_00['const'] = 1
<ipython-input-21-8cecf2c6e250>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_00['age_sq'] = data_00['age'] ** 2


const    -0.293275
hyrsed    0.110333
age       0.084360
age_sq   -0.000887
black    -0.259617
other    -0.062144
dtype: float64

In [22]:
print('The absolute difference between the estimates for alpha for 2000 is ',
      abs(min_results_00['x'][1] - results.params[0]))
print('The absolute difference between the estimates for beta1 for 2000 is ',
      abs(min_results_00['x'][2] - results.params[1]))
print('The absolute difference between the estimates for beta2 for 2000 is ',
      abs(min_results_00['x'][3] - results.params[2]))
print('The absolute difference between the estimates for beta3 for 2000 is ',
      abs(min_results_00['x'][4] - results.params[3]))
print('The absolute difference between the estimates for beta4 for 2000 is ',
      abs(min_results_00['x'][5] - results.params[4]))
print('The absolute difference between the estimates for beta5 for 2000 is ',
      abs(min_results_00['x'][6] - results.params[5]))

The absolute difference between the estimates for alpha for 2000 is  7.411356824791773
The absolute difference between the estimates for beta1 for 2000 is  0.21215322984464197
The absolute difference between the estimates for beta2 for 2000 is  0.22446615195730524
The absolute difference between the estimates for beta3 for 2000 is  0.002705527586852378
The absolute difference between the estimates for beta4 for 2000 is  0.47434900156684356
The absolute difference between the estimates for beta5 for 2000 is  0.014723820936353757


# Comments
1. I must have made a mistake somewhere because the calcualted increase for $\color{red}{\textbf{1980}}$ \& $\color{red}{\textbf{2000}}$ seem unrealistic. However, there is not a lot of difference for the $\beta_{1}$ output between mle and ols.
2. I am consistently off on my alpha coefficient. I expect this has something to do with the constant = 1.

$\textbf{I can't figure out my mistake.}$